In [1]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


#This is what we will change if we want the whole data. Right now it only runs for 44th president

for i in range(43,44):
    x = i
    absolutepath_president = f'/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[{x}]/label'
    president1 = browser.find_element_by_xpath(absolutepath_president)
    president1.click()
    time.sleep(2)


# president = browser.find_element_by_xpath(
#     '/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[42]/label')
# president.click()





SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')    

In [21]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string

link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        


http = urllib3.PoolManager()
list2=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    



#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    if list2[i]:
        list3.append(str(list2[i]))
    else:
        
        #add comments for why I had to do it like this
        
        url = "http://millercenter.org" + new_link_str[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        list3.append(str(soup_2.findAll('div', {"class":"view-transcript"})))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')
    speech_dates[i] = datetime.date(speech_dates[i])


df = pd.DataFrame({'Date':speech_dates, 'Speeches':cleaned_speeches})



Add a function to map from a date to a President

In [22]:
from sympy import Piecewise
import numpy
from sympy.utilities.lambdify import lambdify
from sympy.abc import x
import datetime
import re
def pres_from_date(date_):
    p = lambdify(x, Piecewise(('Theodore_Roosevelt', ((x >= date(1901, 9, 14).toordinal()) & (x < date(1909, 3, 4).toordinal()))),
                              ('William_Howard_Taft', ((x >= date(1909, 3, 4).toordinal()) & (x < date(1913, 3, 4).toordinal()))),
                              ('Woodrow_Wilson', ((x >= date(1913, 3, 4).toordinal()) & (x < date(1921, 3, 4).toordinal()))),
                              ('Warren_G_Harding', ((x >= date(1921, 3, 4).toordinal()) & (x < date(1923, 8, 3).toordinal()))),
                              ('Calvin_Coolidge', ((x >= date(1923, 8, 3).toordinal()) & (x < date(1929, 3, 4).toordinal()))),
                              ('Herbert_Hoover', ((x >= date(1929, 3, 4).toordinal()) & (x < date(1933, 3, 4).toordinal()))),
                              ('Franklin_D_Roosevelt', ((x >= date(1933, 3, 4).toordinal()) & (x < date(1945, 4, 12).toordinal()))),
                              ('Harry_S_Truman', ((x >= date(1945, 4, 12).toordinal()) & (x < date(1953, 1, 20).toordinal()))),
                              ('Dwight_D_Eisenhower', ((x >= date(1953, 1, 20).toordinal()) & (x < date(1961, 1, 20).toordinal()))),
                              ('John_F_Kennedy', ((x >= date(1961, 1, 20).toordinal()) & (x < date(1963, 11, 22).toordinal()))),
                              ('Lyndon_B_Johnson', ((x >= date(1963, 11, 22).toordinal()) & (x < date(1969, 1, 20).toordinal()))),
                              ('Richard_M_Nixon', ((x >= date(1969, 1, 20).toordinal()) & (x < date(1974, 8, 9).toordinal()))),
                              ('Gerald_R_Ford', ((x >= date(1974, 8, 9).toordinal()) & (x < date(1977, 1, 20).toordinal()))),
                              ('Jimmy_Carter', ((x >= date(1977, 1, 20).toordinal()) & (x < date(1981, 1, 20).toordinal()))),
                              ('Ronald_Reagan', ((x >= date(1981, 1, 20).toordinal()) & (x < date(1989, 1, 20).toordinal()))),
                              ('George_H_W_Bush', ((x >= date(1989, 1, 20).toordinal()) & (x < date(1993, 1, 20).toordinal()))),
                              ('William_J_Clinton', ((x >= date(1993, 1, 20).toordinal()) & (x < date(2001, 1, 20).toordinal()))),
                              ('George_W_Bush', ((x >= date(2001, 1, 20).toordinal()) & (x < date(2009, 1, 20).toordinal()))),
                              ('Barack_Obama', ((x >= date(2009, 1, 20).toordinal()) & (x < date(2017, 1, 20).toordinal()))),
                              ('Donald_J_Trump', (x >= date(2017, 1, 20).toordinal()))
                              ),
                              "numpy")
    return re.sub('_', " ", str(p(date_.toordinal())))

In [77]:
## set up data frame
import pandas as pd

df1 = pd.read_csv('Dow_Jones_data_since_1900.csv')
df2 = pd.read_csv('Nasdaq_since_1971.csv')
df3 = pd.read_csv('S&P_data_since_1950.csv')
df1 = df1[df1.Dow_Change != 0]
df1 = df1.drop(columns = 'Close')
df2 = df2.drop(columns = 'Close')
df3 = df3.drop(columns = 'Close')
fin_data = df1.merge(df2, how = "left")
fin_data = fin_data.merge(df3, how = "left")

print(fin_data)

## This function should work if there are no instances where two speeches happened on consecutive days where
def add_speeches_to_fin_data(df, fin_data):
    for i in range(len(df.Date)):
        df.Date[i] = df.Date[i] + datetime.timedelta(days = 1)
    final = fin_data.merge(df, how = "left")
    for i in list(pd.isnull(fin_data.Dow_Change).nonzero()[0]):
        final.Date[i + 1] = final.Date[i]
        final.Speeches[i + 1] = final.Date[i]
    final = final[pd.notna(final.Dow_Change)]  
    return final

             Date  Dow_Change  NASDAQ_Change  SandP_Change
0        1/4/1900     0.00804            NaN           NaN
1        1/5/1900    -0.00660            NaN           NaN
2        1/6/1900    -0.01045            NaN           NaN
3        1/8/1900     0.00587            NaN           NaN
4        1/9/1900    -0.02185            NaN           NaN
5       1/10/1900    -0.01325            NaN           NaN
6       1/11/1900    -0.01375            NaN           NaN
7       1/12/1900     0.02557            NaN           NaN
8       1/13/1900    -0.00201            NaN           NaN
9       1/15/1900    -0.00903            NaN           NaN
10      1/16/1900     0.00109            NaN           NaN
11      1/17/1900     0.00418            NaN           NaN
12      1/18/1900    -0.00436            NaN           NaN
13      1/19/1900     0.01547            NaN           NaN
14      1/20/1900    -0.00710            NaN           NaN
15      1/22/1900     0.00826            NaN           N

In [78]:
a = add_speeches_to_fin_data(df, fin_data)
a[29420:29600]

,Date,Dow_Change,NASDAQ_Change,SandP_Change,speeches
29420,7/23/2008,0.00257,0.0095,0.0041,NaN
29421,7/24/2008,-0.02494,-0.0197,-0.0231,NaN
29422,7/25/2008,0.00188,0.0133,0.0042,NaN
29423,7/28/2008,-0.02153,-0.0200,-0.0186,NaN
29424,7/29/2008,0.02338,0.0245,0.0234,NaN
29425,7/30/2008,0.01607,0.0044,0.0167,NaN
29426,7/31/2008,-0.01808,-0.0018,-0.0131,NaN
29427,8/1/2008,-0.00456,-0.0063,-0.0056,NaN
29428,8/4/2008,-0.00374,-0.0110,-0.0090,NaN
29429,8/5/2008,0.02855,0.0281,0.0287,NaN


In [87]:
from nltk.tokenize import sent_tokenize
sent_tokenize_list = sent_tokenize(text)
len(sent_tokenize_list)
# sent_tokenize_list

177

In [90]:
import nltk
from statistics import mean
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
score = []
sid = SentimentIntensityAnalyzer()
for sentence in sent_tokenize_list:
#    print(sentence)
    ss = sid.polarity_scores(sentence)
    score.append(ss['compound'])

    for k in ss:
        print('{0}: {1}, '.format(k, ss[k]), end='\n')
#     print()
mean(score)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
neg: 0.0, 
neu: 0.545, 
pos: 0.455, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.465, 
pos: 0.535, 
compound: 0.3182, 
neg: 0.0, 
neu: 0.286, 
pos: 0.714, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.667, 
pos: 0.333, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.734, 
pos: 0.266, 
compound: 0.7096, 
neg: 0.102, 
neu: 0.721, 
pos: 0.177, 
compound: 0.25, 
neg: 0.0, 
neu: 0.783, 
pos: 0.217, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.643, 
pos: 0.357, 
compound: 0.7269, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 0.686, 
pos: 0.314, 
compound: 0.4939, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: 0.0, 
neg: 0.0, 
neu: 0.538, 
pos: 0.462, 
compound: 0.8225, 
neg: 0.0, 
neu: 0.667, 
pos: 0.333, 
compound: 0.3612, 
neg: 0.0, 
neu: 0.741, 
pos: 0.259, 
compound: 0.6369, 
neg: 0.179, 
neu: 0.578, 
pos: 0.243, 
compound: 0.2732, 
neg:

0.2080050847457627

In [91]:
text = 'this is a shitty sentence'
ss = sid.polarity_scores(text)
for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='\n')
#compound -1 to 1

neg: 0.522, 
neu: 0.29, 
pos: 0.188, 
compound: -0.5106, 
